In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
import scipy
from tqdm import tqdm
import sys

import bilby

# Add module
sys.path.append('../../modules')
from compute_omega import sample_dict_compute_injected_omega, calculate_num_injections, calculate_total_rate
from gwBackground_module import *

from constants import *
import equations
import redshiftPrior

In [2]:
# Plot formatting
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
fontsize = 24
params = {
   'axes.labelsize': fontsize,
   'font.size': fontsize,
   'legend.fontsize': 8,
   'xtick.labelsize': fontsize,
   'ytick.labelsize': fontsize,
   'axes.titlesize':fontsize,
   'lines.linewidth':1,  
   'xtick.direction':'in',
   'ytick.direction':'in',
   'font.family':'Serif',
   'font.serif':'Hoefler Text',
   'axes.grid':True,
   'figure.figsize': (6.75, 4),
   'figure.dpi':250,
   'mathtext.fontset':'cm'
}

for param in params.keys():
    matplotlib.rcParams[param] = params[param]  

In [1]:
# Constants
# Mass Parameters
BBH_min = 5 # minimum BBH mass
BBH_max = 44 # maximum BBH mass

# Data Simulation Parameters
T_obs = (10/365.25) # observation time (in yrs)
duration = T_obs*60*60*24*365.25 # duration of each segment of data (in s)
fs = 2048 # sampling rate (in Hz)

num_iterations = 50

In [4]:
# Standard Prior Dictionary
priors = bilby.gw.prior.BBHPriorDict(aligned_spin=True)
priors['mass_1'] = bilby.core.prior.PowerLaw(alpha=-2.3, minimum=BBH_min, maximum=BBH_max)
priors['mass_ratio'] = bilby.core.prior.PowerLaw(alpha=1.5, minimum=0, maximum=1)
priors['theta_jn'] = bilby.core.prior.Uniform(minimum=0, maximum=2*np.pi, name='theta_jn')
priors['chi_1'] = 0
priors['chi_2'] = 0
priors['redshift'] = redshiftPrior.BrokenPowerLawRedshiftPrior(R0=R0, alpha=alpha, beta=beta, zp=zp, minimum=0, maximum=z_max, name='redshift')
priors['geocent_time'] = bilby.core.prior.Uniform(minimum=0, maximum=T_obs, name='geocent_time')
del priors['luminosity_distance']
del priors['chirp_mass']

16:06 bilby INFO    : Using aligned spin prior
16:06 bilby INFO    : No prior given, using default BBH priors in /Users/patrickmeyers/opt/anaconda3/envs/gw_general/lib/python3.9/site-packages/bilby/gw/prior_files/aligned_spins_bbh.prior.


In [5]:
# Weighted Method (Waveform Generator)
# Uniform Prior Dictionary
uniform_priors = bilby.gw.prior.BBHPriorDict(aligned_spin=True)
del uniform_priors['chirp_mass']
del uniform_priors['luminosity_distance']
uniform_priors['mass_1'] = bilby.core.prior.Uniform(minimum=BBH_min, maximum=BBH_max, name='mass_1_uniform')
uniform_priors['mass_ratio'] = bilby.core.prior.Uniform(minimum=0, maximum=1, name='mass_ratio_uniform')
# uniform_priors['redshift'] = bilby.gw.prior.UniformComovingVolume(0, z_max, name='redshift')
uniform_priors['redshift'] = redshiftPrior.BrokenPowerLawRedshiftPrior(R0=R0, alpha=alpha, beta=beta, zp=zp, minimum=0, maximum=z_max, name='redshift')
uniform_priors['chi_1'] = 0
uniform_priors['chi_2'] = 0

# Draw samples



zs, p_dz = uniform_priors['redshift']._get_redshift_arrays()
unif_rate = calculate_total_rate(zs, p_dz) # yr^-1
print(unif_rate)
unif_rate = unif_rate / (86400 * 365.25)**2

N = calculate_num_injections(T_obs, zs, p_dz).value
N_inj = np.random.poisson(N)
print(N_inj)
# N_inj = 200_000

injections = uniform_priors.sample(N_inj)



# get Tobs needed to get 100_000 injections at the rate
# from above.
Tobs_fake = N_inj / unif_rate
# Tobs_fake = T_obs * 86400 * 365.25
print(Tobs_fake)

m1s = injections['mass_1']
qs = injections['mass_ratio']
zs = injections['redshift']
m2s = []
Mtots = [] # m1 + m2
Ms = [] # chirp masses
for i in range(len(injections['mass_1'])):
    m2s.append(equations.calculate_m2(m1s[i], qs[i]))
    Mtots.append(equations.calculate_Mtot(m1s[i], m2s[i]))
    Ms.append(equations.calculate_M(m1s[i], m2s[i]))

# Set up waveform generator
# Waveform generator parameters
sampling_frequency=fs
approximant='TaylorF2'

# Initialize waveform generator
waveform_generator = bilby.gw.WaveformGenerator(
    duration=2,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": approximant,
        "reference_frequency": 50,
        "minimum_frequency": 1
    }
)

# Calculate non-weighted dEdf from uniform distributions
freqs_W = waveform_generator.frequency_array
OmegaGW_W = np.zeros(len(freqs_W))

dEdfs = []
for i in tqdm(range(N_inj)):
    inj_params = {}
    # Generate the individual parameters dictionary for each injection
    for k in injections.keys():
        if k == 'signal_type':
            continue
        try:
            inj_params[k] = injections[k]['content'][i]
        except:
            inj_params[k] = injections[k][i]

    # Get frequency domain waveform
    polarizations = waveform_generator.frequency_domain_strain(inj_params)

    # Final PSD of the injection
    dEdf = np.abs(polarizations['plus']) ** 2 + np.abs(polarizations['cross']) ** 2
    dEdfs.append(dEdf / Tobs_fake)
   

16:06 bilby INFO    : Using aligned spin prior
16:06 bilby INFO    : No prior given, using default BBH priors in /Users/patrickmeyers/opt/anaconda3/envs/gw_general/lib/python3.9/site-packages/bilby/gw/prior_files/aligned_spins_bbh.prior.


240463.57826358674 1 / yr
120491


16:06 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


499014583071231.4 yr


100%|███████████████████████████████████████████████████████████| 120491/120491 [00:26<00:00, 4561.17it/s]


In [6]:
%time
# Calculate Omega(f)
dEdf_sum = 0
weights = []
for i in range(N_inj):
    z = zs[i]
    m1 = m1s[i]
    m2 = m2s[i]
    q = qs[i]
    Mtot = Mtots[i]
    M = Ms[i]

    # Probability of drawing {z, m1, m2}
    p_z = priors['redshift'].prob(z)
    p_m1 = priors['mass_1'].prob(m1)
    p_q = priors['mass_ratio'].prob(q)

    pdraw_z = uniform_priors['redshift'].prob(z)
    pdraw_m1 = uniform_priors['mass_1'].prob(m1)
    pdraw_q = uniform_priors['mass_ratio'].prob(q)

    # # Weight calculation
    # r = equations.R_SI(alpha, beta, z, zp, R0)
    # h = equations.Hubble_rate_SI(z, H0, omegaR, omegaM, omegak, omegaL)
    h0 = equations.Hubble_rate_SI(0, H0, omegaR, omegaM, omegak, omegaL)

    wi = (p_z/pdraw_z) * (p_m1/pdraw_m1) * (p_q/pdraw_q)
    weights.append(wi)
    dEdf = wi*dEdfs[i]

    dEdf_sum += dEdf
    
    # Add to Omega_spectrum
    # OmegaGW_W += 2 * np.pi**2 * freqs_W**3 * dEdf / (3 * h0**2)
OmegaGW_W = 2 * np.pi**2 * freqs_W**3 * dEdf_sum / (3 * h0**2)
# Tobs_seconds = T_obs * 86400 * 365.25  # years to seconds
OmegaGW_W *= 2

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 3.81 µs


In [7]:
# omgws = []
# for ii in range(200):
#     freqs_psd, omega_gw_freq, injections = sample_dict_compute_injected_omega(priors, Tobs=T_obs, duration=2, f_ref=25, sampling_frequency=fs,
#                                        approximant=approximant)
#     omgws.append(omega_gw_freq)

In [8]:
# fives, fifties, ninefives = np.percentile(omgws, [5, 50, 95], axis=0)
plt.figure(figsize=(12, 6))
plt.plot(freqs_psd, OmegaGW_W * 86400 * 365.25, c='r', lw=3)
# plt.plot(freqs_psd, fifties, c='k', ls='--')
plt.fill_between(freqs_psd, fives, ninefives, facecolor='k', alpha=0.3)
plt.xscale("log")
plt.yscale("log")
plt.show()

NameError: name 'freqs_psd' is not defined

In [ ]:
plt.hist(np.array([omgws[ii][10] / (OmegaGW_W[10]) for ii in range(len(omgws))]), bins='auto')
plt.show()

In [ ]:
print(np.mean([omgws[ii][10] / (OmegaGW_W[10]) for ii in range(len(omgws))]))